In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password
import psycopg2 as pg2

### Store CSV into DataFrame

In [2]:
#I used Securities.csv & Fundamentals.csv because the both have a common primary key... "Ticker symbol".

csv_file = "securities.csv"
securities = pd.read_csv(csv_file)
securities.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [3]:
#I used Securities.csv & Fundamentals.csv because the both have a common primary key... "Ticker symbol".

csv_file = "fundamentals.csv"
Fund_Stock = pd.read_csv(csv_file)
Fund_Stock.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [4]:
csv_file = "prices.csv"
Stock_Prices = pd.read_csv(csv_file)
Stock_Prices.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Create new data with select columns

In [5]:
#Dropping all other columns and only using these three criteria (Maybe we could use a filter to show which companies have the most equity?)
#I also Added "Cash Ratio". Company's that has 1 Cash Ratio should be able to pay off thier Liabilitys with the cash reserves. (IE.)
# There Might be a coorolation between "Cash Ratio's" and "Total Equity"



# Dataframe 1
Securities_DF_Cleaned = securities[['Ticker symbol', 'Security', 'GICS Sector', 'GICS Sub Industry']]
Securities_DF_Cleaned = Securities_DF_Cleaned.rename(columns={'Ticker symbol': 'Ticker Symbol'})
Securities_DF_Cleaned.head()

# Dataframe 2
Fund_Stock_DF_Cleaned = Fund_Stock[['Ticker Symbol', 'Period Ending', 'Total Equity', 'Cash Ratio', 'Total Revenue', 'For Year']]
Fund_Stock_DF_Cleaned.head()

# Dataframe 3
Stock_Prices_Cleaned = Stock_Prices.rename(columns={"symbol": "Ticker Symbol", "open": "Open", "close": "Close",
                                                   "low": "Low", "high": "High", "volume": "Volume",
                                                   "date": "Date"})
Stock_Prices_Cleaned = Stock_Prices_Cleaned[["Ticker Symbol", "Date", "Open", "Close", "Low", "High", "Volume"]]
Stock_Prices_Cleaned.head()

,Ticker Symbol,Date,Open,Close,Low,High,Volume
0,WLTW,2016-01-05 00:00:00,123.430000,125.839996,122.309998,126.250000,2163600.0
1,WLTW,2016-01-06 00:00:00,125.239998,119.980003,119.940002,125.540001,2386400.0
2,WLTW,2016-01-07 00:00:00,116.379997,114.949997,114.930000,119.739998,2489500.0
3,WLTW,2016-01-08 00:00:00,115.480003,116.620003,113.500000,117.440002,2006300.0
4,WLTW,2016-01-11 00:00:00,117.010002,114.970001,114.089996,117.330002,1408600.0


### Connect to local database

In [6]:
# 1.) You first need to create a dataframe within your local SQL enviroment (TIP: To make it easier name the DF in SQL "NYSE")
# 2.) This Code Pushes Fund_Stock_DF_Cleaned Dataframe into SQL.
# 3.) Also, Please make sure you Change (password="Google123") to your password.
try:
    conn = pg2.connect(dbname="NYSE", user = username, password = password)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE "Securities"(
    "Ticker Symbol" varchar(7) NOT NULL PRIMARY KEY,
    "Security" text,
    "GICS Sector" text,
    "GICS Sub Industry" text);
    """)
    conn.commit()
except:
    print("Unable to Connect")


In [7]:
# 1.) This Code Pushes Fund_Stock_DF_Cleaned Dataframe into SQL.
# 2.) Also, Please make sure you Change (password="Google123") to your password.
try:
    conn = pg2.connect(dbname="NYSE", user = username, password = password)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE "Fund Stock"(
    "Ticker Symbol" varchar(7) REFERENCES "Securities"("Ticker Symbol") ,
    "Period Ending" date,
    "Total Equity" bigint,
    "Cash Ratio" decimal,
    "Total Revenue" bigint,
    "For Year" int);
    """)
    conn.commit()
except:
    print("Unable to Connect")

In [8]:
try:
    conn = pg2.connect(dbname="NYSE", user = username, password = password)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE "Stock Prices"(
    "Ticker Symbol" varchar(7) REFERENCES "Securities"("Ticker Symbol"),
    "Date" date,
    "Open" decimal,
    "Close" decimal,
    "Low" decimal,
    "High" decimal,
    "Volume" decimal);
    """)
    conn.commit()
except:
    print("Unable to Connect")

In [9]:
rds_connection_string = f"{username}:{password}@localhost:5432/NYSE"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [10]:
engine.table_names()

['Stock Prices', 'Securities', 'Fund Stock']

### Use pandas to load converted DataFrames into database

In [11]:
Securities_DF_Cleaned.to_sql(name="Securities", con=engine, if_exists='append', index=False)

Fund_Stock_DF_Cleaned.to_sql(name="Fund Stock", con=engine, if_exists='append', index=False)

Stock_Prices_Cleaned.to_sql(name="Stock Prices", con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the Securities and Fund Stock databases
* NOTE: can also check using pgAdmin